### Data tests

In [139]:
# import os
# from data.alpha import Alpha

# path = os.path.join(os.path.expanduser('~'), "git/bakery/bakery/data/config.json")
# alpha = Alpha(path)

# # Active list
# result = alpha.etl_active_list()

# # Daily
# symbols = ["TSLA", "MSFT", "IBM", "AAPL"]
# result = alpha.etl_daily(symbols=symbols)

# # Hourly
# symbols = ["NVDA", "META"]
# years = list(range(2022, 2024))
# months = list(range(1, 13))
# result = alpha.etl_hourly(symbols=symbols, years=years, months=months)

"Truncate collection". ie. delete all documents, but keep empty collection

In [176]:
# import os
# import json
# from pymongo import MongoClient

# path = os.path.join(os.path.expanduser('~'), "git/bakery/bakery/data/config.json")
# with open(path, "rb") as f:
#     config = json.loads(f.read().decode())
# client = MongoClient(config["mongo"]["conn_str"])

# db = client["bakery"]
# coll = db[""]

# result = coll.delete_many({})

Read from DB

In [141]:
# import os
# import json
# from pymongo import MongoClient
# import pandas as pd

# path = os.path.join(os.path.expanduser('~'), "git/bakery/bakery/data/config.json")
# with open(path, "rb") as f:
#     config = json.loads(f.read().decode())
# client = MongoClient(config["mongo"]["conn_str"])

# db = client["bakery"]
# coll = db["alpha_daily"]

# cursor = coll.aggregate([
#     {
#          "$group": {
#             "_id": "$symbol",
#             "min": {"$min": "$dttm"},
#             "max": {"$max": "$dttm"}
#         }
#     }
# ])

In [142]:
# df = pd.DataFrame(list(cursor))

In [143]:
# df.shape

In [144]:
# df.sort_values(by="_id")

In [145]:
# cursor = coll.find({"symbol": "TSLA"})

In [146]:
# df = pd.DataFrame(list(cursor))
# df.head()

In [147]:
# df.shape

## Data Plots

In [194]:
import os
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient

from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, DatetimeTickFormatter, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [195]:
output_notebook()

Loading BokehJS ...

In [196]:
path = os.path.join(os.path.expanduser('~'), "git/bakery/bakery/data/config.json")
with open(path, "rb") as f:
    config = json.loads(f.read().decode())
client = MongoClient(config["mongo"]["conn_str"])
db = client["bakery"]
alpha_daily_active_ls_coll = db["alpha_daily_active_ls"]

In [208]:
cursor = alpha_daily_active_ls_coll.find({})
df = pd.DataFrame(list(cursor))
df.head()

,dttm,_id,ls
0,2016-01-28,65d2c4b59a0b11a2ecbb7e6e,"[A, AAAP, AABA, AAC, AACG, AADR, AAIC, AAL, AA..."
1,2016-01-29,65d2c4b59a0b11a2ecbb7e6d,"[A, AAAP, AABA, AAC, AACG, AADR, AAIC, AAL, AA..."
2,2016-01-30,65d2c4b29a0b11a2ecbb7e6c,"[A, AAAP, AABA, AAC, AACG, AADR, AAIC, AAL, AA..."
3,2016-01-31,65d2c4ad9a0b11a2ecbb7e6b,"[A, AAAP, AABA, AAC, AACG, AADR, AAIC, AAL, AA..."
4,2016-02-01,65d2c4a79a0b11a2ecbb7e6a,"[A, AAAP, AABA, AAC, AACG, AADR, AAIC, AAL, AA..."


In [209]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   dttm    2944 non-null   datetime64[ns]
 1   _id     2944 non-null   object        
 2   ls      2944 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 69.1+ KB


In [210]:
df_explode = df.drop(columns="_id") \
    .explode(column="ls") \
    .reset_index(drop=True) \
    .rename(columns={"ls": "symbol"})
df_explode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28811775 entries, 0 to 28811774
Data columns (total 2 columns):
 #   Column  Dtype         
---  ------  -----         
 0   dttm    datetime64[ns]
 1   symbol  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 439.6+ MB


In [211]:
df_pivot = df_explode \
    .pivot_table(index="dttm", columns="symbol", aggfunc="size", fill_value=0) \
    .sort_values(by="dttm")
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2944 entries, 2016-01-28 to 2024-02-18
Columns: 18144 entries, -P-S to ZZZ
dtypes: int64(18144)
memory usage: 407.6 MB


In [230]:
# df_pivot.index = df_pivot.index.to_pydatetime()
df_pivot.index.names = ["Date"]
df_pivot.columns.name = "Symbol"
df_pivot.head()

Symbol,-P-S,A,AA,AA-W,AAA,AAAP,AAAU,AABA,AAC,AAC-U,...,ZX,ZXYZ-A,ZXZZT,ZY,ZYME,ZYNE,ZYRX,ZYXI,ZZK,ZZZ
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-28,0,1,0,0,0,1,0,1,1,0,...,1,1,1,0,0,1,1,1,0,1
2016-01-29,0,1,0,0,0,1,0,1,1,0,...,1,1,1,0,0,1,1,1,0,1
2016-01-30,0,1,0,0,0,1,0,1,1,0,...,1,1,1,0,0,1,1,1,0,1
2016-01-31,0,1,0,0,0,1,0,1,1,0,...,1,1,1,0,0,1,1,1,0,1
2016-02-01,0,1,0,0,0,1,0,1,1,0,...,1,1,1,0,0,1,1,1,0,1


In [236]:
dates = list([dt.strftime("%m/%d/%Y") for dt in df_pivot.index])
symbols = list(df_pivot.columns)

In [237]:
df = pd.DataFrame(df_pivot.stack(), columns=["active"]).reset_index()
df.head()

,Date,Symbol,active
0,2016-01-28,-P-S,0
1,2016-01-28,A,1
2,2016-01-28,AA,0
3,2016-01-28,AA-W,0
4,2016-01-28,AAA,0


In [238]:
source = ColumnDataSource(data=df)

colors = ["#bcbd22", "#ff9896"]
mapper = LinearColorMapper(palette=colors, low=0, high=1)

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title="Active Equities Heatmap",
           x_axis_type="datetime",
           x_range=dates, y_range=symbols,
           x_axis_location="above", width=900, height=700,
           tools=TOOLS, toolbar_location="below",
           tooltips=[("date", "@Symbol @Date"), ("active", "@active")])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "4px"
p.axis.major_label_standoff = 0
p.xaxis.formatter = DatetimeTickFormatter(years="%Y", months="%m/%Y", days="%d/%m/%Y")
p.xaxis.major_label_orientation = 3.14 / 3


p.rect(x="Date", y="Symbol", width=1, height=1,
       source=df,
       fill_color={"field": "active", "transform": mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="4px",
                     ticker=BasicTicker(desired_num_ticks=2),
                     label_standoff=6, border_line_color=None)
p.add_layout(color_bar, "right")

AttributeError: unexpected attribute 'format' to DatetimeTickFormatter, possible attributes are days, hourmin, hours, js_event_callbacks, js_property_callbacks, microseconds, milliseconds, minsec, minutes, months, name, seconds, subscribed_events, syncable, tags or years

In [ ]:
show(p)

### Dash tests

In [5]:
# import os
# import json
# import streamlit as st
# from pymongo import MongoClient
# import pandas as pd

# path = os.path.join(os.path.expanduser('~'), "git/bakery/bakery/data/config.json")

# with open(path, "rb") as f:
#     config = json.loads(f.read().decode())

# client = MongoClient(config["mongo"]["conn_str"])
# db = client["bakery"]
# coll = db["alpha_daily"]

# query = {"symbol": "TSLA"}
# df = pd.DataFrame(list(coll.find(query)))

# print(df.head(25))